만약 Colab environment인 경우 다음과 같은 코드 실행

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/2026PredictingBDACompletion/src')

Mounted at /content/drive


Local environment의 경우 cd 명령어를 사용하여 working directory 조정

최종적으로, 다음 코드 조각이 이 파일이 속한 디렉토리를 지시해야 함

In [2]:
print(os.getcwd())

/content/drive/MyDrive/2026PredictingBDACompletion/src


In [3]:
!pip install -r ../requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.7/124.7 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 6.1 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.6.0
    Uninstalling triton-3.6.0:
      Successfully uninstalled triton-3.6.0
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.67.3
    Uninstalling tqdm-4.67.3:
      Successfully uninstalled tqdm-4.67.3
  Attempting uninstall: nvidia-nvshmem-cu12
    Found existing installation: nvidia-nvshmem-cu12 3.4.5
    Uninstalling nvidia-nvshmem-cu12-3.4.

In [4]:
import json
import os
import random
import sys
from datetime import datetime
from pathlib import Path
import numpy as np
import pandas as pd
import optuna
from optuna.samplers import TPESampler
from sklearn.metrics import f1_score, log_loss
from sklearn.model_selection import StratifiedKFold, StratifiedGroupKFold
import xgboost as xgb
import torch

# =======================
# CONFIG
# =======================
BASE_DIR = Path("..")
OUTPUT_DIR = BASE_DIR / "outputs"
SUBMIT_XGBOOST = OUTPUT_DIR / "submission_xgboost.csv"
SUBMIT_CATBOOST = OUTPUT_DIR / "submission_catboost.csv"
SUBMIT_FINAL = BASE_DIR / "submission_final.csv"

GLOBAL_SEED = 42

# =======================
# UTILS
# =======================
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

# =======================
# MAIN
# =======================
def main():
    seed_everything(GLOBAL_SEED)

    print(f"[Info] Trying to load submissions from {OUTPUT_DIR}")
    if not SUBMIT_XGBOOST.exists():
        raise FileNotFoundError(f"[Error] Xgboost submission not found at {SUBMIT_XGBOOST}")
        return
    else:
        try:
            sub_xgb = pd.read_csv(SUBMIT_XGBOOST)
        except:
            raise Exception(f"[Error] Xgboost submission loading failed from {SUBMIT_XGBOOST}")
            return

    if not SUBMIT_CATBOOST.exists():
        raise FileNotFoundError(f"[Error] Catboost submission not found at {SUBMIT_CATBOOST}")
        return
    else:
        try:
            sub_cb = pd.read_csv(SUBMIT_CATBOOST)
        except:
            raise Exception(f"[Error] Catboost submission loading failed from {SUBMIT_CATBOOST}")
            return

    assert sub_cb['ID'].equals(sub_xgb['ID']), "ID mismatch! Check your CSV files."

    print("[Info] Files loaded successfully.")

    sub_ens = (sub_cb['completed'] * 0.5) + (sub_xgb['completed'] * 0.5)
    sub_final = (sub_ens > 0.5).astype(int)

    submission = pd.DataFrame({
        'ID': sub_cb['ID'],
        'completed': sub_final
    })
    submission.to_csv(SUBMIT_FINAL, index=False)

    print(f"[Info] Saved ensemble submission to: {SUBMIT_FINAL}")

if __name__ == "__main__":
    main()

[Info] Trying to load submissions from ../outputs
[Info] Files loaded successfully.
[Info] Saved ensemble submission to: ../submission_final.csv
